In [1]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import math
import random
import numpy as np
from scipy.optimize import minimize

In [218]:
def apply_fixed_ansatz(qubits, parameters, circuit):

    for iz in range (0, len(qubits)):
        circuit.ry(parameters[0][iz], qubits[iz])

    # circ.cz(qubits[0], qubits[1])
    # circ.cz(qubits[2], qubits[0])

    # for iz in range (0, len(qubits)):
    #     circ.ry(parameters[1][iz], qubits[iz])

    # circ.cz(qubits[1], qubits[2])
    # circ.cz(qubits[2], qubits[0])

    # for iz in range (0, len(qubits)):
    #     circ.ry(parameters[2][iz], qubits[iz])

circ = QuantumCircuit(1)
apply_fixed_ansatz([0], [ [1]  ], circ)
circ.draw()

┌───────┐
q: ┤ Ry(1) ├
   └───────┘

In [220]:
# Creates the Hadamard test

def had_test(gate_type, qubits, auxiliary_index, parameters, circ):

    circ.h(auxiliary_index)

    apply_fixed_ansatz(qubits, parameters, circ)
    for first in range(0, len(gate_type)):
        for ie in range (0, len(gate_type[first])):
            if (gate_type[first][ie] == 1):
                circ.cz(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 2):
                # print(2)
                circ.cx(auxiliary_index, qubits[ie])
            elif (gate_type[first][ie] == 3):
                circ.cy(auxiliary_index, qubits[ie])

    # for ie in range (0, len(gate_type[1])):
    #     if (gate_type[1][ie] == 1):
    #         circ.cz(auxiliary_index, qubits[ie])
    #     elif (gate_type[0][ie] == 2):
    #         circ.cx(auxiliary_index, qubits[ie])
    #     elif (gate_type[0][ie] == 3):
    #         circ.cy(auxiliary_index, qubits[ie])
    
    circ.h(auxiliary_index)
    
circ = QuantumCircuit(2)
had_test([ [1] , [2]], [1], 0, [ [1] ], circ)
circ.draw()

# circ = QuantumCircuit(3)
# had_test([ [0,1], [1,0] ], [1, 2], 0, [ [1, 1],])
# circ.draw()

┌───┐          ┌───┐
q_0: ──┤ H ├───■───■──┤ H ├
     ┌─┴───┴─┐ │ ┌─┴─┐└───┘
q_1: ┤ Ry(1) ├─■─┤ X ├─────
     └───────┘   └───┘

In [221]:
# Creates controlled anstaz for calculating |<b|psi>|^2 with a Hadamard test

def control_fixed_ansatz(qubits, parameters, auxiliary, reg, circ):

    for i in range (0, len(qubits)):
        circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    # circ.ccx(auxiliary, qubits[1], 2)
    # circ.cz(qubits[0], 0)
    # circ.ccx(auxiliary, qubits[0], 2)

    # circ.ccx(auxiliary, qubits[0], 4)
    # circ.cz(qubits[0], 2)
    # circ.ccx(auxiliary, qubits[0], 4)

    # circ.ccx(auxiliary, qubits[0], 2)

    # for i in range (0, len(qubits)):
    #     circ.cry(parameters[0][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    # for i in range (0, len(qubits)):
    #     circ.cry(parameters[1][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

    # circ.ccx(auxiliary, qubits[2], 2)
    # circ.cz(qubits[1], 2)
    # circ.ccx(auxiliary, qubits[2], 2)

    # circ.ccx(auxiliary, qubits[0], 4)
    # circ.cz(qubits[2], 4)
    # circ.ccx(auxiliary, qubits[0], 4)

    # for i in range (0, len(qubits)):
    #     circ.cry(parameters[2][i], qiskit.circuit.Qubit(reg, auxiliary), qiskit.circuit.Qubit(reg, qubits[i]))

q_reg = QuantumRegister(3)
circ = QuantumCircuit(q_reg)
control_fixed_ansatz([1], [ [1] ], 0, q_reg, circ)
circ.draw()

q26571_0: ────■────
          ┌───┴───┐
q26571_1: ┤ Ry(1) ├
          └───────┘
q26571_2: ─────────

In [222]:
import cmath
from qiskit.extensions import UnitaryGate

def convert_unitary_to_gate(theta):
    mat = [
        [cmath.cos(theta), cmath.sin(theta)],
        [cmath.sin(theta), -cmath.cos(theta)]
    ]
    return UnitaryGate(mat)

In [223]:
def control_b(auxiliary, qubits):
    u = convert_unitary_to_gate(np.pi/4)
    cb = u.control(1)

    circ.append(cb, [auxiliary, qubits])

    # for ia in qubits:
    #     circ.cx(auxiliary, ia)

circ = QuantumCircuit(2)
control_b(0, [1])
circ.draw()

/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lucyjiao/miniforge3/envs/quantum/lib/python3.11/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


q_0: ─────■─────
     ┌────┴────┐
q_1: ┤ Unitary ├
     └─────────┘

In [224]:
# Create the controlled Hadamard test, for calculating <psi|psi>

def special_had_test(gate_type, qubits, auxiliary_index, parameters, reg, circuit):

    # circ.h(auxiliary_index)
    # # control_b(auxiliary_index, qubits)
    # control_b(auxiliary_index, [2])
    

    # control_fixed_ansatz(qubits, parameters, auxiliary_index, reg)

    # for ty in range (0, len(gate_type)):
    #     if (gate_type[ty] == 1):
    #         circ.cz(auxiliary_index, qubits[ty])
    #     elif (gate_type[ty] == 2):
    #         circ.cx(auxiliary_index, qubits[ty])
    #     elif (gate_type[ty] == 3):
    #         circ.cy(auxiliary_index, qubits[ty])

    # circ.cx(1, 2)
    # circ.h(1)
    # # circ.rz(-np.pi/2, auxiliary_index)
   
    # circ.h(auxiliary_index)
    circuit.h(reg[0]) #Applying a Hadamard gate
    # circuit.x(reg[1]) #Comment to make both states non-orthogonal
    circuit.cswap(reg[0],reg[1],reg[2]) # Contolled SWAP gate
    circuit.h(reg[0]) 

q_reg = QuantumRegister(3)
circ = QuantumCircuit(q_reg)
special_had_test([ 1 ], [1], 0, [ [1] ], q_reg, circ)
circ.draw()

┌───┐   ┌───┐
q26584_0: ┤ H ├─■─┤ H ├
          └───┘ │ └───┘
q26584_1: ──────X──────
                │      
q26584_2: ──────X──────

In [239]:
# coefficient_set = [0.55, 0.45]
# gate_set = [ [0, 0, 0], [0, 0, 1] ]

coefficient_set = [1.5, 0.5]
gate_set = [ [0], [2] ]

# Implements the entire cost function on the quantum circuit

def calculate_cost_function(parameters):
    
    global opt

    overall_sum_1 = 0
    
    # parameters = [parameters[0:1]]
    parameters = [[np.real(parameters)]]


    # print(parameters)

    for i in range(0, len(gate_set)):
        for j in range(0, len(gate_set)):

            qctl = QuantumRegister(3)
            qc = ClassicalRegister(3)
            circ = QuantumCircuit(qctl, qc)

            backend = Aer.get_backend('aer_simulator')
            
            multiply = coefficient_set[i]*coefficient_set[j]

            had_test([gate_set[i], gate_set[j]], [1], 0, parameters, circ)

            circ.save_statevector()
            t_circ = transpile(circ, backend)
            # qobj = assemble(t_circ)
            job = backend.run(t_circ)

            result = job.result()
            outputstate = np.real(result.get_statevector(circ, decimals=100))
            o = outputstate

            m_sum = 0
            for l in range (0, len(o)):
                if (l%2 == 1):
                    n = o[l]**2
                    m_sum+=n

            overall_sum_1+=multiply*(1-(2*m_sum))

    overall_sum_2 = 0

    # for i in range(0, len(gate_set)):
    #     for j in range(0, len(gate_set)):

    #         multiply = coefficient_set[i]*coefficient_set[j]
    #         mult = 1

    #         for extra in range(0, 2):

    #             qctl = QuantumRegister(3)
    #             qc = ClassicalRegister(3)
    #             circ = QuantumCircuit(qctl, qc)

    #             backend = Aer.get_backend('aer_simulator')

    #             if (extra == 0):
    #                 special_had_test(gate_set[i], [1], 0, parameters, qctl)
    #             if (extra == 1):
    #                 special_had_test(gate_set[j], [1], 0, parameters, qctl)

    #             circ.save_statevector()    
    #             t_circ = transpile(circ, backend)
    #             # qobj = assemble(t_circ)
    #             job = backend.run(t_circ)

    #             result = job.result()
    #             outputstate = np.real(result.get_statevector(circ, decimals=100))
    #             o = outputstate

    #             m_sum = 0
    #             for l in range (0, len(o)):
    #                 if (l%2 == 1):
    #                     n = o[l]**2
    #                     m_sum+=n
    #             mult = mult*(1-(2*m_sum))

    #         overall_sum_2+=multiply*mult
    for i in range(0, len(gate_set)):
        
        qctl = QuantumRegister(3)
        qc = ClassicalRegister(3)
        circ = QuantumCircuit(qctl, qc)
        backend = Aer.get_backend('aer_simulator')

        apply_fixed_ansatz([1], parameters, circ)
        # encode a applied to ansantz
        if (gate_set[i] == 1):
            circ.z(1)
        elif (gate_set[i] == 2):
            circ.x(1)
        elif (gate_set[i] == 3):
            circ.y(1)

        # unitary for b 
        u = convert_unitary_to_gate(1*np.pi/4)
        circ.append(u, [2])
        # circ.h(2)

        circ.h(0) #Applying a Hadamard gate
        # circuit.x(reg[1]) #Comment to make both states non-orthogonal
        circ.cswap(0,1,2) # Contolled SWAP gate
        circ.h(0) 
        circ.save_statevector()    
        t_circ = transpile(circ, backend)
        job = backend.run(t_circ)
        result = job.result()
        outputstate = np.real(result.get_statevector(circ, decimals=100))
        o = outputstate

        overall_sum_2 += coefficient_set[i] * o[0]**2

        print(overall_sum_1, overall_sum_2, 1-float(overall_sum_2/overall_sum_1))
            
    # print(1-float(overall_sum_2/overall_sum_1))

    return 1 - float(overall_sum_2/overall_sum_1)

In [234]:
# def calculate_cost_function(parameters):
#     global opt
    
#     parameters = [[np.real(parameters)]]
#     print("parameters: ", parameters)


#     circ = QuantumCircuit(1, 1)
#     apply_fixed_ansatz([0], parameters, circ)
#     circ.save_statevector()
#     circ.measure_all(add_bits=False)

#     backend = Aer.get_backend('aer_simulator')
#     t_circ = transpile(circ, backend)
#     # print(circ)

#     job = backend.run(t_circ, shots=8192)

#     result = job.result()
#     # print(result.get_counts())
#     o = result.get_statevector(circ, decimals=10)
#     # print(o)

#     x_sol_not_normalized = np.array([o[0], o[1]])
#     x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

#     classical_sol = (0.46742, 0.88403)

#     f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
#     # print(x_sol_not_normalized, x_normalized)
#     print("fidelity: ", f2)
#     return (1 - f2)

In [253]:
from scipy.optimize import minimize_scalar

# out = minimize(calculate_cost_function, x0=[[np.pi/2]], method="BFGS", options={'maxiter':200,  "ftol":0.000000000001})
out = minimize_scalar(calculate_cost_function, method="Brent", options={'maxiter':200,  "ftol":0.000000000001})

# out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/10000 for i in range(0, 1)], method="COBYLA", options={'maxiter':200})

print(out)

out_f = [[out['x']]]




/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_70399/3463599535.py:4: OptimizeWarning: Unknown solver options: ftol
  out = minimize_scalar(calculate_cost_function, method="Brent", options={'maxiter':200,  "ftol":0.000000000001})


2.5 0.7500000000000002 0.7
2.5 1.0000000000000002 0.5999999999999999
3.762206477211844 0.5776133647005525 0.8464695204265824
3.762206477211844 0.77015115293407 0.7952926939021099
1.0016732371545203 0.3572924596905318 0.6433043766792632
1.0016732371545203 0.4763899462540424 0.524405835572351
1.0910181406292927 0.5036395448443693 0.5383765621405039
1.0910181406292927 0.6715193931258259 0.384502082854005
1.4727216629591444 0.6482557431059974 0.5598246706010591
1.4727216629591444 0.8643409908079965 0.4130995608014122
1.2044567598261362 0.5640052434523654 0.5317347519110776
1.2044567598261362 0.7520069912698205 0.3756463358814367
1.186298583941129 0.5560021735789588 0.53131346432885
1.186298583941129 0.7413362314386117 0.37508461910513324
1.1777508585285994 0.5520694086859271 0.5312511091049897
1.1777508585285994 0.7360925449145695 0.3750014788066529
1.1763316678241273 0.5514054538546683 0.5312500131237574
1.1763316678241273 0.7352072718062245 0.3750000174983431
1.1764658891060935 0.5514683

In [254]:
out_f

[[-1.080839000899056]]

In [255]:
def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

In [257]:
circ = QuantumCircuit(1, 1)
apply_fixed_ansatz([0], out_f, circ)
circ.save_statevector()
circ.measure_all(add_bits=False)

backend = Aer.get_backend('aer_simulator')
t_circ = transpile(circ, backend)
print(circ)

job = backend.run(t_circ, shots=8192)

result = job.result()
print(result.get_counts())
o = result.get_statevector(circ, decimals=10)
print(o)

a1 = coefficient_set[1]*np.array([ [1,0], [0,-1] ])
a2 = coefficient_set[0]*np.array([ [1,0], [0,1]])
# a2x = coefficient_set[0]*np.array([ [0,1,0,0,0,0,0,0], [1,0,0,0,0,0,0,0], [0,0,0,1,0,0,0,0], [0,0,1,0,0,0,0,0], [0,0,0,0,0,1,0,0], [0,0,0,0,1,0,0,0], [0,0,0,0,0,0,0,1], [0,0,0,0,0,0,1,0] ])

a3 = np.add(a2, a1)

b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

# b = np.array([float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8)),float(1/np.sqrt(8))])

# print((b.dot(a3.dot(o)/(np.linalg.norm(a3.dot(o))))))

# print(o[0], o[1], o[0]/o[1])

x_sol_not_normalized = np.array([o[0]**2, o[1]**2])
x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

classical_sol = (1, 1)

f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
print(x_sol_not_normalized, x_normalized)
print("fidelity: ", f2)



     ┌─────────────┐ statevector  ░ ┌─┐
  q: ┤ Ry(-1.0808) ├──────░───────░─┤M├
     └─────────────┘      ░       ░ └╥┘
c: 1/════════════════════════════════╩═
                                     0 
{'1': 2163, '0': 6029}
Statevector([ 0.85749293+0.j, -0.51449576+0.j],
            dims=(2,))
[0.73529412+0.j 0.26470588-0.j] [0.94088741+0.j 0.33871947-0.j]
fidelity:  (0.9048187023275959+0j)


In [26]:
from qiskit import IBMQ
# load IBM account
IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
IBMQ.load_account()

provider = IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-education', group='harvard', project='qse-210')

/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_52622/2447371092.py:3: DeprecationWarning: The package qiskit.providers.ibmq is being deprecated. Please see https://ibm.biz/provider_migration_guide to get instructions on how to migrate to qiskit-ibm-provider (https://github.com/Qiskit/qiskit-ibm-provider) and qiskit-ibm-runtime (https://github.com/Qiskit/qiskit-ibm-runtime).
  IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
/var/folders/07/x8mtl3_53h95v_q_0ddbc_fh0000gn/T/ipykernel_52622/2447371092.py:3: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qisk

In [40]:
def extract_counts(dict):
    total_counts = 0
    relevant_count0 = 0
    relevant_count1 = 0
    for key in dict:
        if key[0] == '1':
            if key[-1] == '1':
                relevant_count1 += int(dict[key])
            if key[-1] == "0":
                relevant_count0 += int(dict[key])
        total_counts += int(dict[key])
    return (relevant_count0/total_counts, relevant_count1/total_counts)

def extract_solution(probs):
    return (np.sqrt(probs[0]), np.sqrt(probs[1]))

def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

In [44]:
def paper_no2_hardware():
    global circ

    coefficient_set = [0.5, 1.5]
    gate_set = [ [2, 0, 0], [0, 0, 0] ]
    backend=provider.get_backend("ibmq_quito")
    
    out = minimize(calculate_cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 9)], method="COBYLA", options={'maxiter':200})
    print(out)

    out_f = [out['x'][0:1] ]

    circ = QuantumCircuit(1, 1)
    apply_fixed_ansatz([0], out_f)
    circ.measure_all()

    t_circ = transpile(circ, backend)
    print(circ)

    job = backend.run(t_circ, shots=8192, job_tags=["quito-1.5-.5-b=maxmixed-t=pi/2"])

    result = job.result()
    print(result.get_counts())
    o = result.get_counts(circ)
    print(o)
    print(o["0 0"],o["1 0"], o["0 0"]/o["1 0"])

    x_sol_not_normalized = np.array([o["0 0"]/o["1 0"], 1])
    x_normalized = x_sol_not_normalized/np.linalg.norm(x_sol_not_normalized)

    a = np.array([[1.5, 0.5], [0.5, 1.5]])

    b = np.array([float(1/np.sqrt(2)), float(1/np.sqrt(2))])

    classical_sol = (0.354, 0.354)

    f2 = calculate_normalized_fidelity(classical_sol, x_normalized)
    print(x_sol_not_normalized, x_normalized)
    print("fidelity: ", f2)
    print("params: ", out_f)

for _ in range(5):
    paper_no2_hardware()


 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -98.99999999977787
       x: [ 3.142e+00  4.815e-01  6.420e-01  1.551e+00  1.501e+00
            4.026e-01 -4.525e-01  6.926e-01  7.325e-01]
    nfev: 96
   maxcv: 0.0
        ┌────────────┐ ░ ┌─┐
     q: ┤ Ry(3.1416) ├─░─┤M├
        └────────────┘ ░ └╥┘
   c: 1/══════════════════╬═
                          ║ 
meas: 1/══════════════════╩═
                          0 
{'0 0': 612, '1 0': 7580}
{'0 0': 612, '1 0': 7580}
612 7580 0.08073878627968338
[0.08073879 1.        ] [0.08047691 0.99675647]
fidelity:  0.7617190288354545
params:  [array([3.14159235])]
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -98.99998222201677
       x: [ 3.142e+00  1.490e+00  2.294e+00  1.313e+00  7.157e-01
            1.762e+00  1.216e+00  1.853e+00 -6.114e-03]
    nfev: 98
   maxcv: 0.0
        ┌────────────┐ ░ ┌─┐
     q: ┤ Ry(3.1415) ├─░─┤M├
        └────────────┘ ░ └╥┘
   c: 1/══════

IBMQBackendApiError: 'Error submitting job: "(\'Connection aborted.\', RemoteDisconnected(\'Remote end closed connection without response\'))"'